# How to pass config to tools

You may need to pass values to a tool that are only known at runtime. For example, the tool logic may require using the ID of the user who made the request.

Most of the time, such values should not be controlled by the LLM. In fact, allowing the LLM to control the user ID may lead to a security risk.

Instead, the LLM should only control the parameters of the tool that are meant to be controlled by the LLM, while other parameters (such as user ID) should be fixed by the application logic.

To pass run time information, we will use tools that leverage the LangChain Runnable interface. The standard runnables methods (invoke, batch, stream etc.) accept a 2nd argument which is a RunnableConfig. RunnableConfig has a few standard fields, but allows users to use other fields for run time information.

Here, we will show how to set up a simple agent that has access to three tools for saving, reading, and deleting a list of the user's favorite pets.

## Setup

First we need to install the packages required

```bash
npm install @langchain/langgraph @langchain/openai
```

Next, we need to set API keys for OpenAI (the LLM we will use)

```bash
export OPENAI_API_KEY=your-api-key
```

Optionally, we can set API key for [LangSmith tracing](https://smith.langchain.com/), which will give us best-in-class observability.

```bash
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_CALLBACKS_BACKGROUND="true"
export LANGCHAIN_API_KEY=your-api-key
```

## Defining Tools and Model

In [16]:
import { tool } from '@langchain/core/tools';
import { z } from 'zod';

let userToPets = {};

const updateFavoritePets = tool(
async (input, config) => {
    let userId = config?.configurable.user_id;
    userToPets[userId] = input.pets;
    return "update_favorite_pets called.";
},
{
    name: "update_favorite_pets",
    description: "add to the list of favorite pets.",
    schema: z.object({
    pets: z.array(z.string()),
    }),
}
);

const deleteFavoritePets = tool(
async (_, config) => {
    let userId = config?.configurable.user_id;
    if (userId in userToPets) {
    delete userToPets[userId];
    }
    return "delete_favorite_pets called.";
},
{
    name: "delete_favorite_pets",
    description: "Delete the list of favorite pets.",
    schema: z.object({}),
}
);

const listFavoritePets = tool(
async (_, config) => {
    let userId = config?.configurable.user_id;
    return JSON.stringify(userToPets[userId] ?? []);
},
{
    name: "list_favorite_pets",
    description: "List favorite pets if any.",
    schema: z.object({}),
}
);

let tools = [updateFavoritePets, deleteFavoritePets, listFavoritePets];

We'll be using a small chat model from Anthropic in our example. To use chat models with tool calling, we need to first ensure that the model is aware of the available tools. We do this by calling `.bindTools` method on `ChatAnthropic` moodel

In [17]:
import { ChatAnthropic } from "@langchain/anthropic";

let model = new ChatAnthropic({ model: "claude-3-haiku-20240307", temperature: 0});
let modelWithTools = model.bindTools(tools);

## ReAct Agent

Let's set up a graph implementation of the [ReAct agent](https://langchain-ai.github.io/langgraphjs/concepts/agentic_concepts/#react-agent). This agent takes some query as input, then repeatedly call tools until it has enough information to resolve the query. We'll be using prebuilt `ToolNode` and the Anthropic model with tools we just defined

In [18]:
import { END, START, StateGraph } from "@langchain/langgraph";
import { AIMessage } from "@langchain/core/messages";
import { RunnableConfig } from "@langchain/core/runnables";
import { Annotation } from "@langchain/langgraph";
import { BaseMessage } from "@langchain/core/messages";
import { ToolNode } from "@langchain/langgraph/prebuilt";

const AgentState = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: (x, y) => x.concat(y),
  }),
});

const routeMessage = (state: typeof AgentState.State) => {
  const { messages } = state;
  const lastMessage = messages[messages.length - 1] as AIMessage;
  // If no tools are called, we can finish (respond to the user)
  if (!lastMessage?.tool_calls?.length) {
    return END;
  }
  // Otherwise if there is, we continue and call the tools
  return "tools";
};

const callModel = async (
  state: typeof AgentState.State,
  config?: RunnableConfig,
) => {
  // For versions of @langchain/core < 0.2.3, you must call `.stream()`
  // and aggregate the message from chunks instead of calling `.invoke()`.
  const { messages } = state;
  const responseMessage = await modelWithTools.invoke(messages, config);
  return { messages: [responseMessage] };
};

const workflow = new StateGraph(AgentState)
  .addNode("agent", callModel)
  .addNode("tools", new ToolNode(tools))
  .addEdge(START, "agent")
  .addConditionalEdges("agent", routeMessage)
  .addEdge("tools", "agent");

const graph = workflow.compile();

## Use it!

Let's use our graph now!

In [19]:
console.log(`User information prior to run: ${JSON.stringify(userToPets)}`);

let inputs = { messages: [["user", "my favorite pets are cats and dogs"]] };
let config = {"configurable": {"user_id": "123"}};

for await (
  const chunk of await graph.stream(inputs, {
    ...config,
  })
) {
  for (const [node, values] of Object.entries(chunk)) {
    console.log(`Output from node: ${node}`);
    console.log("---");
    console.log(values);
    console.log("\n====\n");
  }
}

console.log(`User information after run: ${JSON.stringify(userToPets)}`);

User information prior to run: {}
Output from node: agent
---
{
  messages: [
    AIMessage {
      "id": "msg_01AiCXUt95qndBeU41UzTUF1",
      "content": [
        {
          "type": "text",
          "text": "Okay, let's update your favorite pets list:"
        },
        {
          "type": "tool_use",
          "id": "toolu_01PoPA9dBDPkdct7f6SrWbQu",
          "name": "update_favorite_pets",
          "input": {
            "pets": "[Array]"
          }
        }
      ],
      "additional_kwargs": {
        "id": "msg_01AiCXUt95qndBeU41UzTUF1",
        "type": "message",
        "role": "assistant",
        "model": "claude-3-haiku-20240307",
        "stop_reason": "tool_use",
        "stop_sequence": null,
        "usage": {
          "input_tokens": 509,
          "output_tokens": 71
        }
      },
      "response_metadata": {
        "id": "msg_01AiCXUt95qndBeU41UzTUF1",
        "model": "claude-3-haiku-20240307",
        "stop_reason": "tool_use",
        "stop_sequence":

In [20]:
console.log(`User information prior to run: ${JSON.stringify(userToPets)}`);

inputs = { messages: [["user", "what are my favorite pets?"]] };

for await (
  const chunk of await graph.stream(inputs, {
    ...config,
  })
) {
  for (const [node, values] of Object.entries(chunk)) {
    console.log(`Output from node: ${node}`);
    console.log("---");
    console.log(values);
    console.log("\n====\n");
  }
}

console.log(`User information after run: ${JSON.stringify(userToPets)}`);

User information prior to run: {"123":["cats","dogs"]}
Output from node: agent
---
{
  messages: [
    AIMessage {
      "id": "msg_017Vnq5GQgtXAD297ThkVsSe",
      "content": [
        {
          "type": "text",
          "text": "Let me check your favorite pets:"
        },
        {
          "type": "tool_use",
          "id": "toolu_01Q6rJyyHJHBxYMzKbPMttgi",
          "name": "list_favorite_pets",
          "input": {}
        }
      ],
      "additional_kwargs": {
        "id": "msg_017Vnq5GQgtXAD297ThkVsSe",
        "type": "message",
        "role": "assistant",
        "model": "claude-3-haiku-20240307",
        "stop_reason": "tool_use",
        "stop_sequence": null,
        "usage": {
          "input_tokens": 508,
          "output_tokens": 46
        }
      },
      "response_metadata": {
        "id": "msg_017Vnq5GQgtXAD297ThkVsSe",
        "model": "claude-3-haiku-20240307",
        "stop_reason": "tool_use",
        "stop_sequence": null,
        "usage": {
       

In [21]:
console.log(`User information prior to run: ${JSON.stringify(userToPets)}`);

inputs = { messages: [["user", "please forget what i told you about my favorite animals"]] };

for await (
  const chunk of await graph.stream(inputs, {
    ...config,
  })
) {
  for (const [node, values] of Object.entries(chunk)) {
    console.log(`Output from node: ${node}`);
    console.log("---");
    console.log(values);
    console.log("\n====\n");
  }
}

console.log(`User information after run: ${JSON.stringify(userToPets)}`);

User information prior to run: {"123":["cats","dogs"]}
Output from node: agent
---
{
  messages: [
    AIMessage {
      "id": "msg_01QV8qhDDCZ4dmx6dt8cUCzP",
      "content": [
        {
          "type": "tool_use",
          "id": "toolu_01BixxrjGVyR5vfr25LHo2vp",
          "name": "delete_favorite_pets",
          "input": {}
        }
      ],
      "additional_kwargs": {
        "id": "msg_01QV8qhDDCZ4dmx6dt8cUCzP",
        "type": "message",
        "role": "assistant",
        "model": "claude-3-haiku-20240307",
        "stop_reason": "tool_use",
        "stop_sequence": null,
        "usage": {
          "input_tokens": 512,
          "output_tokens": 38
        }
      },
      "response_metadata": {
        "id": "msg_01QV8qhDDCZ4dmx6dt8cUCzP",
        "model": "claude-3-haiku-20240307",
        "stop_reason": "tool_use",
        "stop_sequence": null,
        "usage": {
          "input_tokens": 512,
          "output_tokens": 38
        },
        "type": "message",
      